# EMR Data Prep + SageMaker Deep Learning

This notebook is tested using `Studio SparkMagic - PySpark Kernel` running on a `ml.t3.medium` instance and connected to an EMR clsuter with an `m5.xlarge` Master node and 2 `m5.xlarge` Core nodes. Please ensure that you see `PySpark (SparkMagic)` in the top right on your notebook.

In this 3 part notebook lesson, we'll see how to use EMR for data prep and serialization to S3. Next we'll prototype a deep learning architecture using SageMaker Studio notebooks, and lastly we'll scale the training using SageMaker ephemeral training jobs.

In [ ]:
# %load_ext sagemaker_studio_analytics_extension.magics
# %sm_analytics emr connect --cluster_id j-xxxxxxxxxxxx --auth-type None 

## Inspect the public NYC Taxi Dataset

In [2]:
%%local
!aws s3 ls "s3://nyc-tlc/trip data/green" --human-readable | grep green_tripdata_2016

2016-08-12 12:07:19  220.3 MiB green_tripdata_2016-01.csv
2016-08-12 12:08:24  230.2 MiB green_tripdata_2016-02.csv
2016-08-12 12:09:18  240.3 MiB green_tripdata_2016-03.csv
2016-08-12 12:04:43  235.5 MiB green_tripdata_2016-04.csv
2016-08-12 12:05:31  234.5 MiB green_tripdata_2016-05.csv
2016-08-12 12:06:26  214.2 MiB green_tripdata_2016-06.csv
2017-01-17 19:33:38  116.6 MiB green_tripdata_2016-07.csv
2017-01-17 19:34:13  109.1 MiB green_tripdata_2016-08.csv
2017-01-17 19:31:30  101.6 MiB green_tripdata_2016-09.csv
2017-01-17 19:31:58  109.5 MiB green_tripdata_2016-10.csv
2017-01-17 19:32:30  100.3 MiB green_tripdata_2016-11.csv
2017-01-17 19:33:07  106.8 MiB green_tripdata_2016-12.csv


In [3]:
df = spark.read.csv("s3://nyc-tlc/trip data/green_tripdata_2016*.csv", header=True, inferSchema=True, timestampFormat='yyyy-MM-dd HH:mm:ss').cache()
df.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

16385532

## Format the dataset

In [4]:
%%pretty
from pyspark.sql.functions import col, dayofweek, month, hour
df_dt = df.select(dayofweek(col('lpep_pickup_datetime')).alias('day_of_week'),
                   month(col('lpep_pickup_datetime')).alias('month'),
                   hour(col('lpep_pickup_datetime')).alias('hour'),
                   col("Pickup_latitude").alias("pickup_latitude"),
                   col("Pickup_longitude").alias("pickup_longitude"),
                   col("Dropoff_latitude").alias("dropoff_latitude"),
                   col("Dropoff_latitude").alias("dropoff_longitude"),
                   col("Trip_distance").alias("trip_distance"),
                   col("Fare_amount").alias("fare_amount")
                  )
df_dt.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

day_of_week,month,hour,pickup_latitude,pickup_longitude,dropoff_latitude,dropoff_longitude,trip_distance,fare_amount
6,1,0,40.680610656738281,-73.928642272949219,40.698043823242188,40.698043823242188,1.46,8
6,1,0,40.723175048828125,-73.952674865722656,40.761379241943359,40.761379241943359,3.56,15.5
6,1,0,40.676105499267578,-73.971611022949219,40.646072387695313,40.646072387695313,3.79,16.5
6,1,0,40.669578552246094,-73.989501953125,40.689033508300781,40.689033508300781,3.01,13.5
6,1,0,40.682853698730469,-73.964729309082031,40.663013458251953,40.663013458251953,2.55,12
6,1,0,40.746456146240234,-73.891143798828125,40.742111206054688,40.742111206054688,1.37,7
6,1,0,40.746196746826172,-73.896675109863281,40.745689392089844,40.745689392089844,.57,5
6,1,0,40.803558349609375,-73.953353881835937,40.794120788574219,40.794120788574219,1.01,7
6,1,0,40.702816009521484,-73.994064331054688,40.679725646972656,40.679725646972656,2.46,12
6,1,0,40.756641387939453,-73.914131164550781,40.739658355712891,40.739658355712891,1.61,9


## Run Data Clean Up at Scale on the Cluster

In [5]:
df_dt = df_dt[
    (df_dt.fare_amount > 0)
    & (df_dt.fare_amount < 200)    
]
df_dt.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

8973394

In [8]:
%%pretty
df_dt = df_dt[
    (df_dt.pickup_latitude != 0)    
]
df_dt.show()
df_dt.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

day_of_week,month,hour,pickup_latitude,pickup_longitude,dropoff_latitude,dropoff_longitude,trip_distance,fare_amount
6,1,0,40.680610656738281,-73.928642272949219,40.698043823242188,40.698043823242188,1.46,8
6,1,0,40.723175048828125,-73.952674865722656,40.761379241943359,40.761379241943359,3.56,15.5
6,1,0,40.676105499267578,-73.971611022949219,40.646072387695313,40.646072387695313,3.79,16.5
6,1,0,40.669578552246094,-73.989501953125,40.689033508300781,40.689033508300781,3.01,13.5
6,1,0,40.682853698730469,-73.964729309082031,40.663013458251953,40.663013458251953,2.55,12
6,1,0,40.746456146240234,-73.891143798828125,40.742111206054688,40.742111206054688,1.37,7
6,1,0,40.746196746826172,-73.896675109863281,40.745689392089844,40.745689392089844,.57,5
6,1,0,40.803558349609375,-73.953353881835937,40.794120788574219,40.794120788574219,1.01,7
6,1,0,40.702816009521484,-73.994064331054688,40.679725646972656,40.679725646972656,2.46,12
6,1,0,40.756641387939453,-73.914131164550781,40.739658355712891,40.739658355712891,1.61,9


In [9]:
train_df, val_df = df_dt.randomSplit([0.8, 0.2], seed=42)
val_df, test_df = val_df.randomSplit([0.05, 0.95], seed=42)

print("Train Count:", train_df.count())
print("Validation Count:", val_df.count())
print("Test Count:", test_df.count())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Train Count: 7169617
Validation Count: 89392
Test Count: 1700967

In [10]:
%%local 
import sagemaker

sess = sagemaker.Session()
role = sagemaker.get_execution_role()
bucket = sess.default_bucket()

data_bucket = f"{bucket}/nyc-taxi/data/processed"
print(data_bucket)

sagemaker-us-west-1-176842773820/nyc-taxi/data/processed


In [11]:
%%send_to_spark -i data_bucket -t str -n data_bucket

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Successfully passed 'data_bucket' as 'data_bucket' to Spark kernel

In [12]:
train_df.write.csv(f"s3://{data_bucket}/train", mode='overwrite')
test_df.write.csv(f"s3://{data_bucket}/test", mode='overwrite')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Store data location for next notebook

In [13]:
%store data_bucket

Stored 'data_bucket' (str)
